In [ ]:
import signal
import requests
from time import sleep
import numpy as np
import sys

class ApiException(Exception):
    pass

def signal_handler(sigum, frame):
    global shutdown 
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown=True
    
API_KEY = {'X-API-Key': 'YA6YJ9TB'}
shutdown=False

s = requests.session() 
s.headers.update(API_KEY)

#Global Variables
SPEEDBUMP = 0.5
MAX_LONG_EXPOSURE_NET = 25000
MAX_SHORT_EXPOSURE_NET = -25000
MAX_EXPOSURE_GROSS = 500000
ORDER_LIMIT = 5000
MAX_ORDERS = 5
ticker_list = ['RGLD','RFIN','INDX']

def get_tick(s):   
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick'], case['status']
    raise ApiException('Authorization error, check key')


def get_bid_ask(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        bid_side_book = book['bids']
        ask_side_book = book['asks']
        
        bid_prices_book = [item["price"] for item in bid_side_book]
        ask_prices_book = [item['price'] for item in ask_side_book]
        
        best_bid_price = bid_prices_book[0]
        best_ask_price = ask_prices_book[0]
  
        return best_bid_price, best_ask_price

def get_time_sales(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/securities/tas', params = payload)
    if resp.ok:
        book = resp.json()
        time_sales_book = [item["quantity"] for item in book]
        return time_sales_book

def get_position():
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        book = resp.json()
        gross_position = abs(book[0]['position']) + abs(book[1]['position']) + abs(book[2]['position'])
        net_position = book[0]['position'] + book[1]['position'] + book[2]['position']
        return gross_position, net_position

def get_open_orders(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/orders', params = payload)
    if resp.ok:
        orders = resp.json()
        buy_orders = [item for item in orders if item["action"] == "BUY"]
        sell_orders = [item for item in orders if item["action"] == "SELL"]
        return buy_orders, sell_orders
    
def open_sells(s):
    resp = s.get('http://localhost:9999/v1/orders?status=OPEN')
    if resp.ok:
        open_sells_volume = 0  #total combined volume of all open sells
        ids = []  #all open sell ids
        prices = []  #all open sell prices
        order_volumes = []  #all open sell volumes
        volume_filled = []  #volume filled for each open sell order
     
        open_orders = resp.json()
        for order in open_orders:
            if order['action'] == 'SELL':
                volume_filled.append(order['quantity_filled'])
                order_volumes.append(order['quantity'])
                open_sells_volume = open_sells_volume + order['quantity']
                prices.append(order['price'])
                ids.append(order['order_id'])
                
        return volume_filled, open_sells_volume, ids, prices, order_volumes

def open_buys(s):
    resp=s.get('http://localhost:9999/v1/orders?status=OPEN')
    if resp.ok:
        open_buys_volume = 0 
        ids = []
        prices = [] 
        order_volumes = []
        volume_filled = []
         
        open_orders = resp.json()
        for order in open_orders:
            if order[ 'action'] == 'BUY':
                open_buys_volume = open_buys_volume + order['quantity']
                volume_filled.append(order['quantity_filled'])
                order_volumes.append(order['quantity'])
                prices.append(order['price'])
                ids.append(order['order_id'])
     
        return volume_filled, open_buys_volume, ids, prices, order_volumes

def buy_sell(s, sell_price, buy_price, ticker_symbol):
    for i in range(MAX_ORDERS):
        s.post('http://localhost:9999/v1/orders',params={'ticker': ticker_symbol, 'type': 'LIMIT',
        'quantity': ORDER_LIMIT, 'price': sell_price, 'action': 'SELL'})
        s.post('http://localhost:9999/v1/orders', params={'ticker': ticker_symbol, 'type': 'LIMIT', 
        'quantity': ORDER_LIMIT, 'price': buy_price, 'action': 'BUY'})
        
def re_order(s, number_of_orders, ids, volumes_filled, volumes, price, action):
    for i in range(number_of_orders):
        id = ids[i]
        volume = volumes[i]
        volume_filled = volumes_filled[i]
        #if the order is partially filled.
        if(volume_filled != 0):
            volume = ORDER_LIMIT - volume_filled
            #delete then re-purcnase.
            deleted = s.delete('http://localhost:9999/v1/orders/{)'.format(id))
            if(deleted.ok):
                s.post('http://localhost:9999/v1/orders', params ={'ticker': ticker_symbol,
                'type': 'LIMIT', 'quantity': volume, 'price': price, 'action': action})

def get_order_status(order_id):
    resp = s.get('http://localhost:9999/v1/orders' + '/' + str(order_id))
    if resp.ok:
        order = resp.json()
        return order['status']
    

def main():
    #instantiate variables about all the open buy orders
    buy_ids = []
    buy_prices = []
    buy_volumes = []
    volume_filled_buys = []
    open_buys_volume = 0
     
    #instantiate variables about all the open sell orders
    sell_ids = []
    sell_prices = []
    sell_volumes = []
    volume_filled_sells = []
    open_sells_volume = 0
     
    #instantiated variables when just one side of the book has been completely filled
    single_side_filled = False
    single_side_transaction_time = 0
    #creates a session to manage connections and requests to the ril client
    tick, status = get_tick(s)
    ticker_list = ['RGLD','RFIN','INDX']
    market_prices = np.array([0.,0.,0.,0.,0.,0.])
    market_prices = market_prices.reshape(3,2)
 
    while (tick > 5 and tick < 295 and not shutdown) and status == 'ACTIVE':
        for i in range(3):
            ticker_symbol = ticker_list[i]
            market_prices[i,0], market_prices[i,1] = get_bid_ask(ticker_symbol)

            gross_position, net_position = get_position()
            
            print(f'{(market_prices[0, 0] + market_prices[1, 0]) - market_prices[2, 1]}')
                  
            if gross_position < MAX_EXPOSURE_GROSS and net_position > MAX_SHORT_EXPOSURE_NET and net_position < MAX_LONG_EXPOSURE_NET:
                if market_prices[0, 0] + market_prices[1, 0] > market_prices[2, 1]: 
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RGLD', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[0, 1], 'action': 'SELL'})
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RFIN', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[1, 1], 'action': 'SELL'})
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'INDX', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[2, 0], 'action': 'BUY'})
                    
                if market_prices[0, 1] + market_prices[1, 1] < market_prices[2, 0]: 
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RGLD', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[0, 0], 'action': 'BUY'})
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RFIN', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[1, 0], 'action': 'BUY'})
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'INDX', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[2, 1], 'action': 'SELL'})
                    
            sleep(SPEEDBUMP) 

        tick, status = get_tick(s)

if __name__ == '__main__':
    main()

